In [11]:
import cv2
import numpy as np
import glob

#7 by 7 checkboard
squares_x = 7 
squares_y = 7
square_length = 0.03   # meters
marker_length = 0.015   # meters


#Charuco setup
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50) #50 unique marker
#define the board physically and create 3d reference model
board = cv2.aruco.CharucoBoard(
    (squares_x, squares_y),
    square_length,
    marker_length,
    aruco_dict
)

#Detectors (NEW API) tune the parameter 
aruco_detector = cv2.aruco.ArucoDetector(aruco_dict) #dectect square marker 
charuco_detector = cv2.aruco.CharucoDetector(board) #refine marker corncer

#Storage
all_corners = [] #2d image point
all_ids = [] #IDs of detected corner
image_size = None #image resolution

#Load images
images = glob.glob("calib-images/*.jpg") #load image from file
print("Total images found:", len(images)) #detect how many image in file

#read each image
for fname in images:
    img = cv2.imread(fname)
    if img is None:
        continue

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #convert to grayscale 
    image_size = gray.shape[::-1] #store image size

    #Detect ArUco markers
    corners, ids, _ = aruco_detector.detectMarkers(gray) 

    #reject image with poor dectection
    if ids is None or len(ids) < 4:
        print("Markers NOT found in:", fname)
        continue

    #Detect Charuco corners
    result = charuco_detector.detectBoard(gray)
    charuco_corners, charuco_ids = result[0], result[1]

    #requires enough corncer for dectection 
    if charuco_ids is None or len(charuco_ids) < 15:
        print("Too few ChArUco corners in:", fname)
        continue

    all_corners.append(charuco_corners)
    all_ids.append(charuco_ids)

cv2.destroyAllWindows()

print("Valid images used:", len(all_corners))



#Calibrate camera
objpoints = [] #store 3d real-world point
imgpoints = [] #store 2d image point

for charuco_corners, charuco_ids in zip(all_corners, all_ids):
    #convert corncer into real-world coor (meters) and pixel coor (pixel)
    objp, imgp = board.matchImagePoints(charuco_corners, charuco_ids)
    if objp is None or imgp is None:
        continue
    objpoints.append(objp)
    imgpoints.append(imgp)

#K → intrinsic camera matrix
#dist → distortion coefficients
#rvecs, tvecs → camera pose per image
# #ret → RMS reprojection error
ret, K, dist, rvecs, tvecs = cv2.calibrateCamera(
    objpoints,
    imgpoints,
    image_size,
    None,
    None
)

print("\nCALIBRATION RESULTS")
print("RMS Reprojection Error:", ret)
print("\nCamera Matrix (K):\n", K)
print("\nDistortion Coefficients:\n", dist)




Total images found: 13
Valid images used: 13

CALIBRATION RESULTS
RMS Reprojection Error: 2.5495790139348102

Camera Matrix (K):
 [[4.45331117e+03 0.00000000e+00 2.13938648e+03]
 [0.00000000e+00 4.44501586e+03 2.80138297e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]

Distortion Coefficients:
 [[ 0.30124698 -0.98410235 -0.0044414  -0.00304503  0.55911903]]


In [12]:
import cv2
import numpy as np

#Load image
img = cv2.imread("box.jpg")
if img is None:
    raise RuntimeError("Could not load image")

#Camera intrinsics
K = np.array([
    [4453.31204, 0.0, 2139.38652],
    [0.0, 4445.01674, 2801.38299],
    [0.0, 0.0, 1.0]
], dtype=np.float64)
#fy, fx - focal length(pixel)
#cy, cx prinicipal point (image center) image resoltion - 4284 X 5712 

#radial distortion (k1, k2, k3) and tangential distortion (p1, p2)
#use to correct len warping
dist = np.array([[0.30124582, -0.98409408, -0.00444141, -0.00304501, 0.55909717]],
                dtype=np.float64)

fx = K[0, 0]
fy = K[1, 1]

# Undistort image
#remove barrel distortion and pincushion distortion
undistorted = cv2.undistort(img, K, dist)

#manual pixel coordinates
x1, y1 = 1353, 1894   # top-left corner from ms paint
x2, y2 = 1629, 2113   # bottom-right corner from ms paint

# Pixel dimensions
w = abs(x2 - x1) #weight in pixel
h = abs(y2 - y1) #height in pixel

print(f"Pixel width:  {w} px")
print(f"Pixel height: {h} px")

#Known distance (meters)
Z = 2.5 #distance from center of surface of box to the camera len

#Perspective projection from pinehole camera
#w,h pixel dims, Z - depth, fx,fy - focal length convert pixels to meters 
real_width  = (w * Z) / fx
real_height = (h * Z) / fy

print("\nEstimated real-world dimensions:")
print(f"Width:  {real_width:.3f} meters")
print(f"Height: {real_height:.3f} meters")







Pixel width:  276 px
Pixel height: 219 px

Estimated real-world dimensions:
Width:  0.155 meters
Height: 0.123 meters
